<a href="https://colab.research.google.com/github/rajaranjith/HCL-GenAI-Training/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install SpeechRecognition==3.10.1 gTTS==2.5.3 googletrans==4.0.0-rc1 pydub==0.25.1 ipywidgets==8.1.2


In [ ]:

# Install required packages
!pip install SpeechRecognition==3.10.1 gTTS==2.5.3 googletrans==4.0.0-rc1 pydub==0.25.1 ipywidgets==8.1.2

# For audio handling and UI
import os
import io
import base64
import time
from datetime import datetime

import speech_recognition as sr
from gtts import gTTS
from googletrans import Translator
from pydub import AudioSegment

from IPython.display import Audio, display, HTML, clear_output
import ipywidgets as widgets

# Initialize translator
translator = Translator()

# Language choices
INPUT_LANGUAGES = {
    "English": "en-US",
    "Spanish": "es-ES",
    "French": "fr-FR",
    "German": "de-DE",
    "Hindi": "hi-IN",
}

OUTPUT_LANGUAGES = {
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Japanese": "ja",
}

# Helper: ensure an output folder
os.makedirs("outputs", exist_ok=True)
print("✅ Setup complete.")


✅ Setup complete.


In [ ]:

from google.colab import output

js_code = """
const recordButton = document.createElement('button');
recordButton.textContent = '🎙️ Start Recording';
recordButton.style = 'padding:10px 16px; font-size:16px; margin-right:10px;';
document.body.appendChild(recordButton);

const stopButton = document.createElement('button');
stopButton.textContent = '⏹️ Stop Recording';
stopButton.style = 'padding:10px 16px; font-size:16px;';
stopButton.disabled = true;
document.body.appendChild(stopButton);

let mediaRecorder;
let audioChunks = [];

recordButton.onclick = async () => {
  audioChunks = [];
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  mediaRecorder = new MediaRecorder(stream);
  mediaRecorder.start();
  recordButton.disabled = true;
  stopButton.disabled = false;
  console.log('Recording started');
  mediaRecorder.ondataavailable = e => audioChunks.push(e.data);
};

stopButton.onclick = async () => {
  mediaRecorder.stop();
  stopButton.disabled = true;
  recordButton.disabled = false;
  console.log('Recording stopped');
  mediaRecorder.onstop = async () => {
    const blob = new Blob(audioChunks, { type: 'audio/webm' });
    const reader = new FileReader();
    reader.readAsDataURL(blob);
    reader.onloadend = () => {
      const base64data = reader.result.split(',')[1];
      google.colab.kernel.invokeFunction('notebook.saveAudio', [base64data], {});
    };
  };
};
"""

# Python-side handler to save audio from JS
def save_audio(b64_data):
    audio_bytes = base64.b64decode(b64_data)
    fname = "recorded_audio.webm"
    with open(fname, "wb") as f:
        f.write(audio_bytes)
    print(f"💾 Saved: {fname} ({len(audio_bytes)} bytes)")
    display(HTML(f"<p>✅ Audio saved: <code>{fname}</code></p>"))

output.register_callback('notebook.saveAudio', save_audio)
display(HTML("<h4>Use the buttons below to record your voice:</h4>"))
output.eval_js(js_code)


MessageError: DataCloneError: Failed to execute 'postMessage' on 'MessagePort': async () => {
  mediaRecorder.stop();
  stopButton.disabled = true;
  recordButton.disabled = false;
  console....<omitted>...
} could not be cloned.

In [ ]:

# Convert the recorded webm to wav using pydub (ffmpeg is available in Colab)
src_path = "recorded_audio.webm"
wav_path = "recorded_audio.wav"

assert os.path.exists(src_path), "Please record audio first (see the previous cell)."

# Load and export
audio = AudioSegment.from_file(src_path, format="webm")
audio = audio.set_channels(1)  # mono
audio = audio.set_frame_rate(16000)  # 16 kHz recommended
audio.export(wav_path, format="wav")

print(f"🔁 Converted to WAV: {wav_path}")
display(Audio(wav_path))


🔁 Converted to WAV: recorded_audio.wav


In [ ]:

input_lang_dd = widgets.Dropdown(
    options=list(INPUT_LANGUAGES.keys()),
    value="English",
    description="Input:",
    disabled=False,
)

output_lang_dd = widgets.Dropdown(
    options=list(OUTPUT_LANGUAGES.keys()),
    value="English",
    description="Output:",
    disabled=False,
)

display(widgets.HBox([input_lang_dd, output_lang_dd]))


In [ ]:

# Recognize speech from WAV
recognizer = sr.Recognizer()
input_lang_code = INPUT_LANGUAGES[input_lang_dd.value]
output_lang_code = OUTPUT_LANGUAGES[output_lang_dd.value]

print(f"🎛️ Input language: {input_lang_dd.value} ({input_lang_code})")
print(f"🎛️ Output language: {output_lang_dd.value} ({output_lang_code})")

with sr.AudioFile("recorded_audio.wav") as source:
    audio_data = recognizer.record(source)

try:
    # Speech to Text (using Google's free web API)
    print("🗣️ Recognizing speech...")
    text = recognizer.recognize_google(audio_data, language=input_lang_code)
    print(f"📜 Recognized text: {text}")

    # Translate
    print("🌍 Translating...")
    translated = translator.translate(text, src=input_lang_code.split('-')[0], dest=output_lang_code).text
    print(f"🔤 Translated text: {translated}")

    # TTS
    print("🔊 Synthesizing speech...")
    tts = gTTS(translated, lang=output_lang_code)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_mp3 = f"outputs/translated_{output_lang_dd.value}_{timestamp}.mp3"
    tts.save(out_mp3)
    print(f"✅ Saved TTS: {out_mp3}")

    display(Audio(out_mp3, autoplay=True))

except Exception as e:
    print("❌ Error:", e)


🎛️ Input language: English (en-US)
🎛️ Output language: Japanese (ja)
🗣️ Recognizing speech...
📜 Recognized text: hello
🌍 Translating...
🔤 Translated text: こんにちは
🔊 Synthesizing speech...
✅ Saved TTS: outputs/translated_Japanese_20251203_064319.mp3
